In [1]:
#Optimizacion Bayesiana de hiperparametros de  rpart
#funciona automaticamente con EXPERIMENTOS
#va generando incrementalmente salidas para kaggle

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

library("data.table")
library("rlist")
library("yaml")

library("rpart")
library("parallel")

#paquetes necesarios para la Bayesian Optimization
library("DiceKriging")
library("mlrMBO")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,492757,26.4,1086987,58.1,641780,34.3
Vcells,941618,7.2,8388608,64.0,1753993,13.4


Warning message:
“package ‘data.table’ was built under R version 3.6.3”
Warning message:
“package ‘rlist’ was built under R version 3.6.3”
Warning message:
“package ‘yaml’ was built under R version 3.6.3”
Warning message:
“package ‘rpart’ was built under R version 3.6.3”
Warning message:
“package ‘DiceKriging’ was built under R version 3.6.3”
Warning message:
“package ‘mlrMBO’ was built under R version 3.6.3”
Loading required package: mlr

Warning message:
“package ‘mlr’ was built under R version 3.6.3”
Loading required package: ParamHelpers

Warning message:
“package ‘ParamHelpers’ was built under R version 3.6.3”
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Warning message:
“package ‘smoof’ was built under R version 3.6.3”
Loading required package: 

In [2]:
#para poder usarlo en la PC y en la nube
switch ( Sys.info()[['sysname']],
         Windows = { directory.root   <-  "M:\\" },   #Microsoft Windows
         Darwin  = { directory.root   <-  "~/dm/" },  #Apple MAC
         Linux   = { directory.root   <-  "~/buckets/b1/crudo/" }  #Entorno Google Cloud
       )
#defino la carpeta donde trabajo
setwd( directory.root )

In [3]:
kexperimento  <- NA   #NA si se corre la primera vez, un valor concreto si es para continuar procesando

kscript           <- "04.05 - BO Rpart FE"
karch_generacion  <- "./datasets/paquete_premium_202011_ext.csv"
karch_aplicacion  <- "./datasets/paquete_premium_202101_ext.csv"
kBO_iter    <-  200   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 2000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )


ksemilla_azar  <- 102191

In [4]:
#------------------------------------------------------------------------------
#Funcion que lleva el registro de los experimentos

get_experimento  <- function()
{
  if( !file.exists( "./maestro.yaml" ) )  cat( file="./maestro.yaml", "experimento: 1000" )

  exp  <- read_yaml( "./maestro.yaml" )
  experimento_actual  <- exp$experimento

  exp$experimento  <- as.integer(exp$experimento + 1)
  Sys.chmod( "./maestro.yaml", mode = "0644", use_umask = TRUE)
  write_yaml( exp, "./maestro.yaml" )
  Sys.chmod( "./maestro.yaml", mode = "0444", use_umask = TRUE) #dejo el archivo readonly

  return( experimento_actual )
}

In [5]:
#------------------------------------------------------------------------------
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0(  folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}

In [6]:
#------------------------------------------------------------------------------
#funcion para particionar, es la que Andres reemplaza con caret

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------

In [7]:
ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing, fold==2
  prediccion  <- predict( modelo, data[ fold==fold_test, ], type = "prob")

  prob_baja2  <- prediccion[, "BAJA+2"]

  ganancia_testing  <- sum(  data[ fold==fold_test ][ prob_baja2 >0.025,  ifelse( clase_ternaria=="BAJA+2", 48750, -1250 ) ] )

  return( ganancia_testing )
}

In [8]:
#------------------------------------------------------------------------------

ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )
  particionar( data, divi, seed=semilla, agrupa=pagrupa )

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5  
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 5 )   #se puede subir a 5 si posee Linux o Mac OS

  data[ , fold := NULL ]
  #devuelvo la primer ganancia y el promedio
  return( mean( unlist( ganancias )) *  qfolds )   #aqui normalizo la ganancia
}

In [9]:
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <-  ArbolesCrossValidation( dataset, param=x, qfolds= xval_folds, pagrupa="clase_ternaria", semilla=ksemilla_azar )

   #si tengo una ganancia superadora, genero el archivo para Kaggle
   if(  ganancia > GLOBAL_ganancia_max )
   {
     GLOBAL_ganancia_max <<-  ganancia  #asigno la nueva maxima ganancia
    
     modelo  <- rpart("clase_ternaria ~ .",
                      data= dataset,
                      xval= 0,
                      control= x )

     #genero el vector con la prediccion, la probabilidad de ser positivo
     prediccion  <- predict( modelo, dapply)

     prob_baja2  <- prediccion[, "BAJA+2"]
     Predicted   <- ifelse( prob_baja2 > 0.025, 1, 0 )

     entrega  <-  as.data.table( list( "numero_de_cliente"=dapply$numero_de_cliente, "Predicted"=Predicted)  )

     #genero el archivo para Kaggle
     fwrite( entrega, 
             file= paste0(kkaggle, GLOBAL_iteracion, ".csv" ),
             sep=  "," )
   }

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   loguear( xx,  arch= klog )


   return( ganancia )
}
#------------------------------------------------------------------------------

In [10]:
#Aqui empieza el programa

if( is.na(kexperimento ) )   kexperimento <- get_experimento()  #creo el experimento

In [11]:
#en estos archivos quedan los resultados
kbayesiana  <- paste0("./work/E",  kexperimento, "_rpart.RDATA" )
klog        <- paste0("./work/E",  kexperimento, "_rpart_log.txt" )
kkaggle     <- paste0("./kaggle/E",kexperimento, "_rpart_kaggle_" )

In [12]:
GLOBAL_ganancia_max  <-  -Inf
GLOBAL_iteracion  <- 0

In [13]:
if( file.exists(klog) )
{
 tabla_log  <- fread( klog)
 GLOBAL_iteracion  <- nrow( tabla_log ) -1
 GLOBAL_ganancia_max  <-  tabla_log[ , max(ganancia) ]
}

In [14]:
#cargo los datasets
dataset  <- fread(karch_generacion)   #donde entreno
dapply  <- fread(karch_aplicacion)    #donde aplico el modelo

In [15]:
#Aqui comienza la configuracion de la Bayesian Optimization

configureMlr( show.learner.output = FALSE)

funcion_optimizar  <- EstimarGanancia

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= kbayesiana)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <-  makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

In [ ]:
#inicio la optimizacion bayesiana
if(!file.exists(kbayesiana)) {
  run  <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else  run  <- mboContinue( kbayesiana )   #retomo en caso que ya exista

Computing y column(s) for design. Not provided.



20210910 174809	-0.0729451099337893	3771	1671	14	5	8015000
20210910 174850	-0.394344384167925	4081	1474	9	5	8202500
20210910 174935	-0.946936997158627	2443	425	9	5	8572500
20210910 174959	-0.780221707651799	6280	831	11	5	8397500
20210910 175025	-0.291758705549	5223	952	20	5	8326250
20210910 175052	-0.880817802913953	3492	1313	18	5	8045000
20210910 175112	-0.670646816478984	2528	1174	7	5	8247500
20210910 175137	-0.168921466500615	4719	1759	16	5	7963750
20210910 175149	-0.609347469644854	7307	156	4	5	7923750
20210910 175231	-0.58237784133089	6701	32	19	5	7270000
20210910 175255	-0.79792634487967	7762	1937	13	5	7592500
20210910 175305	0.069039669787162	5809	1608	3	5	0
20210910 175322	-0.641121614130679	4344	1994	6	5	7831250
20210910 175346	-0.259775277442532	1596	471	8	5	8337500
20210910 175417	-0.83355741102132	3217	772	19	5	8327500
20210910 175431	0.014208721718751	7209	1402	16	5	0


[mbo] 0: cp=-0.0729; minsplit=3771; minbucket=1671; maxdepth=14 : y = 8.02e+06 : 48.5 secs : initdesign

[mbo] 0: cp=-0.394; minsplit=4081; minbucket=1474; maxdepth=9 : y = 8.2e+06 : 40.8 secs : initdesign

[mbo] 0: cp=-0.947; minsplit=2443; minbucket=425; maxdepth=9 : y = 8.57e+06 : 44.8 secs : initdesign

[mbo] 0: cp=-0.78; minsplit=6280; minbucket=831; maxdepth=11 : y = 8.4e+06 : 24.8 secs : initdesign

[mbo] 0: cp=-0.292; minsplit=5223; minbucket=952; maxdepth=20 : y = 8.33e+06 : 25.7 secs : initdesign

[mbo] 0: cp=-0.881; minsplit=3492; minbucket=1313; maxdepth=18 : y = 8.04e+06 : 27.0 secs : initdesign

[mbo] 0: cp=-0.671; minsplit=2528; minbucket=1174; maxdepth=7 : y = 8.25e+06 : 19.4 secs : initdesign

[mbo] 0: cp=-0.169; minsplit=4719; minbucket=1759; maxdepth=16 : y = 7.96e+06 : 24.9 secs : initdesign

[mbo] 0: cp=-0.609; minsplit=7307; minbucket=156; maxdepth=4 : y = 7.92e+06 : 12.9 secs : initdesign

[mbo] 0: cp=-0.582; minsplit=6701; minbucket=32; maxdepth=19 : y = 7.27e+0

20210910 175459	-0.226763924877118	2922	1459	20	5	8192500


[mbo] 1: cp=-0.227; minsplit=2922; minbucket=1459; maxdepth=20 : y = 8.19e+06 : 25.3 secs : infill_ei



...

[mbo] 124: cp=-0.683; minsplit=1617; minbucket=793; maxdepth=5 : y = 9.22e+06 : 16.0 secs : infill_ei



20210910 184738	-0.721096978207366	1689	843	5	5	8790000


[mbo] 125: cp=-0.721; minsplit=1689; minbucket=843; maxdepth=5 : y = 8.79e+06 : 15.6 secs : infill_ei



20210910 184756	-0.647273119990605	1530	755	5	5	9321250


[mbo] 126: cp=-0.647; minsplit=1530; minbucket=755; maxdepth=5 : y = 9.32e+06 : 15.4 secs : infill_ei



In [ ]:
quit( save="no" )